In [2]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier
#from pytorch_tabnet.tab_model import TabNetModel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import statsmodels.api as sm
import warnings
import shap
import optuna
import torch
from sklearn import svm
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from tqdm.auto import tqdm
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from pycaret.regression import *
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import ExtraTreeRegressor
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import KNNImputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA

In [3]:
warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', None)

In [4]:
train_data = pd.read_csv("./train.csv")

In [5]:
train_data['YEAR'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.year
train_data['MONTH'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.month
train_data['DAY'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.day
train_data['HOUR'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.hour
train_data['MINUTE'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.minute
train_data['DAY_OF_WEEK'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.day_name()

train_data = train_data.drop(['MONTH', 'DAY', 'YEAR', 'MINUTE', 'HOUR'], axis = 1)

In [6]:
colls = []
for coll in train_data :
    if 'X' in coll and train_data[coll].isna().sum() == 598 :
        colls.append(coll)
print(len(colls))

train_data.drop(colls, inplace = True, axis = 1)

82


In [7]:
train_data.loc[train_data.PRODUCT_CODE == 'A_31','PRODUCT_CODE'] = 1
train_data.loc[train_data.PRODUCT_CODE == 'T_31','PRODUCT_CODE'] = 2
train_data.loc[train_data.PRODUCT_CODE == 'O_31','PRODUCT_CODE'] = 3

train_data.loc[train_data.DAY_OF_WEEK == 'Monday','DAY_OF_WEEK'] = 1
train_data.loc[train_data.DAY_OF_WEEK == 'Tuesday','DAY_OF_WEEK'] = 2
train_data.loc[train_data.DAY_OF_WEEK == 'Wednesday','DAY_OF_WEEK'] = 3
train_data.loc[train_data.DAY_OF_WEEK == 'Thursday','DAY_OF_WEEK'] = 4
train_data.loc[train_data.DAY_OF_WEEK == 'Friday','DAY_OF_WEEK'] = 5
train_data.loc[train_data.DAY_OF_WEEK == 'Saturday','DAY_OF_WEEK'] = 6
train_data.loc[train_data.DAY_OF_WEEK == 'Sunday','DAY_OF_WEEK'] = 7

train_data.drop(['PRODUCT_ID', 'TIMESTAMP'], axis = 1, inplace = True)
train_data

Y_Class  Y_Quality     LINE PRODUCT_CODE   X_1   X_2  X_3   X_4   X_5  \
0          1   0.533433  T050304            1   NaN   NaN  NaN   NaN   NaN   
1          2   0.541819  T050307            1   NaN   NaN  NaN   NaN   NaN   
2          1   0.531267  T050304            1   NaN   NaN  NaN   NaN   NaN   
3          2   0.537325  T050307            1   NaN   NaN  NaN   NaN   NaN   
4          1   0.531590  T050304            1   NaN   NaN  NaN   NaN   NaN   
..       ...        ...      ...          ...   ...   ...  ...   ...   ...   
593        1   0.526546  T100306            2   2.0  95.0  0.0  45.0  10.0   
594        0   0.524022  T050304            1   NaN   NaN  NaN   NaN   NaN   
595        0   0.521289  T050304            1   NaN   NaN  NaN   NaN   NaN   
596        1   0.531375  T100304            3  40.0  94.0  0.0  45.0  11.0   
597        1   0.533702  T100306            3  21.0  87.0  0.0  45.0  10.0   

     X_6   X_7   X_8   X_9  X_10   X_11   X_12   X_13  X_14  X_15   X_16  \
0    NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
1    NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
2    NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
3    NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
4    NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
..   ...   ...   ...   ...   ...    ...    ...    ...   ...   ...    ...   
593  0.0  50.0  10.0  52.0   2.0  468.9  473.9  459.2   0.0   0.0  247.5   
594  NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
595  NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN    NaN   
596  0.0  45.0  10.0  31.0   2.0  505.8  510.9  497.1   0.0   0.0  247.5   
597  0.0  61.0  10.0  52.0   2.0  470.6  474.9  462.9   0.0   0.0  247.5   

      X_17   X_18   X_19   X_20   X_21   X_22  X_23  X_24  X_25   X_26   X_27  \
0      NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
1      NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
2      NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
3      NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
4      NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
..     ...    ...    ...    ...    ...    ...   ...   ...   ...    ...    ...   
593  248.0  247.3  250.0  527.0  538.0  522.0  0.19  0.19  0.19  155.0  155.0   
594    NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
595    NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN    NaN   
596  248.1  247.3  250.0  489.0  498.0  485.0  0.19  0.19  0.19  155.0  155.0   
597  248.0  247.3  250.0  525.0  534.0  520.0  0.19  0.20  0.19  155.0  155.0   

      X_28   X_29   X_30   X_31   X_32   X_33   X_34   X_35   X_36   X_37  \
0      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
1      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
..     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
593  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0  155.0   
594    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
595    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
596  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0  155.0   
597  155.0  155.0  185.0  185.0  185.0  185.0  155.0  155.0  155.0  155.0   

      X_38  X_39  X_40  X_41  X_42  X_43  X_44  X_45  X_46  X_47  X_48  X_49  \
0      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2      NaN   NaN 

In [8]:
T0503041 = train_data[(train_data['LINE'] == 'T050304') & (train_data['PRODUCT_CODE'] == 1)]
T0503042 = train_data[(train_data['LINE'] == 'T050304') & (train_data['PRODUCT_CODE'] == 2)]
T0503043 = train_data[(train_data['LINE'] == 'T050304') & (train_data['PRODUCT_CODE'] == 3)]

T0503071 = train_data[(train_data['LINE'] == 'T050307') & (train_data['PRODUCT_CODE'] == 1)]
T0503072 = train_data[(train_data['LINE'] == 'T050307') & (train_data['PRODUCT_CODE'] == 2)]
T0503073 = train_data[(train_data['LINE'] == 'T050307') & (train_data['PRODUCT_CODE'] == 3)]

T0103051 = train_data[(train_data['LINE'] == 'T010305') & (train_data['PRODUCT_CODE'] == 1)]
T0103052 = train_data[(train_data['LINE'] == 'T010305') & (train_data['PRODUCT_CODE'] == 2)]
T0103053 = train_data[(train_data['LINE'] == 'T010305') & (train_data['PRODUCT_CODE'] == 3)]

T0103061 = train_data[(train_data['LINE'] == 'T010306') & (train_data['PRODUCT_CODE'] == 1)]
T0103062 = train_data[(train_data['LINE'] == 'T010306') & (train_data['PRODUCT_CODE'] == 2)]
T0103063 = train_data[(train_data['LINE'] == 'T010306') & (train_data['PRODUCT_CODE'] == 3)]

T1003041 = train_data[(train_data['LINE'] == 'T100304') & (train_data['PRODUCT_CODE'] == 1)]
T1003042 = train_data[(train_data['LINE'] == 'T100304') & (train_data['PRODUCT_CODE'] == 2)]
T1003043 = train_data[(train_data['LINE'] == 'T100304') & (train_data['PRODUCT_CODE'] == 3)]

T1003061 = train_data[(train_data['LINE'] == 'T100306') & (train_data['PRODUCT_CODE'] == 1)]
T1003062 = train_data[(train_data['LINE'] == 'T100306') & (train_data['PRODUCT_CODE'] == 2)]
T1003063 = train_data[(train_data['LINE'] == 'T100306') & (train_data['PRODUCT_CODE'] == 3)]

In [9]:
T0503041_col = []
for col in T0503041 :
    if 'X' in col and T0503041[col].isna().sum() == len(T0503041) :
        T0503041_col.append(col)
print(len(T0503041_col))

T0503041.drop(T0503041_col, inplace = True, axis = 1)

825


In [10]:
T0503042_col = []
for col in T0503042 :
    if 'X' in col and T0503042[col].isna().sum() == len(T0503042) :
        T0503042_col.append(col)
print(len(T0503042_col))

T0503042.drop(T0503042_col, inplace = True, axis = 1)

2793


In [11]:
T0503071_col = []
for col in T0503071 :
    if 'X' in col and T0503071[col].isna().sum() == len(T0503071) :
        T0503071_col.append(col)
print(len(T0503071_col))

T0503071.drop(T0503071_col, inplace = True, axis = 1)

818


In [12]:
T0103051_col = []
for col in T0103051 :
    if 'X' in col and T0103051[col].isna().sum() == len(T0103051) :
        T0103051_col.append(col)
print(len(T0103051_col))

T0103051.drop(T0103051_col, inplace = True, axis = 1)

1907


In [13]:
T0503071_col = []
for col in T0503071 :
    if 'X' in col and T0503071[col].isna().sum() == len(T0503071) :
        T0503071_col.append(col)
print(len(T0503071_col))

T0503071.drop(T0503071_col, inplace = True, axis = 1)

0


In [14]:
T0103061_col = []
for col in T0103061 :
    if 'X' in col and T0103061[col].isna().sum() == len(T0103061) :
        T0103061_col.append(col)
print(len(T0103061_col))

T0103061.drop(T0103061_col, inplace = True, axis = 1)

1907


In [15]:
T1003042_col = []
for col in T1003042 :
    if 'X' in col and T1003042[col].isna().sum() == len(T1003042) :
        T1003042_col.append(col)
print(len(T1003042_col))

T1003042.drop(T1003042_col, inplace = True, axis = 1)

2122


In [16]:
T1003043_col = []
for col in T1003043 :
    if 'X' in col and T1003043[col].isna().sum() == len(T1003043) :
        T1003043_col.append(col)
print(len(T1003043_col))

T1003043.drop(T1003043_col, inplace = True, axis = 1)

2132


In [17]:
T1003062_col = []
for col in T1003062 :
    if 'X' in col and T1003062[col].isna().sum() == len(T1003062) :
        T1003062_col.append(col)
print(len(T1003062_col))

T1003062.drop(T1003062_col, inplace = True, axis = 1)

2122


In [18]:
T1003063_col = []
for col in T1003063 :
    if 'X' in col and T1003063[col].isna().sum() == len(T1003063) :
        T1003063_col.append(col)
print(len(T1003063_col))

T1003063.drop(T1003063_col, inplace = True, axis = 1)

2132


In [19]:
T0503041x_col = []
for col in T0503041 :
    if 'X' in col and T0503041[col].std() == 0 :
        T0503041x_col.append(col)
print(len(T0503041x_col))
T0503041.drop(T0503041x_col, inplace = True, axis = 1)

T0503071x_col = []
for col in T0503071 :
    if 'X' in col and T0503071[col].std() == 0 :
        T0503071x_col.append(col)
print(len(T0503071x_col))
T0503071.drop(T0503071x_col, inplace = True, axis = 1)

T0103051x_col = []
for col in T0103051 :
    if 'X' in col and T0103051[col].std() == 0 :
        T0103051x_col.append(col)
print(len(T0103051x_col))
T0103051.drop(T0103051x_col, inplace = True, axis = 1)

T0103061x_col = []
for col in T0103061 :
    if 'X' in col and T0103061[col].std() == 0 :
        T0103061x_col.append(col)
print(len(T0103061x_col))
T0103061.drop(T0103061x_col, inplace = True, axis = 1)

T1003042x_col = []
for col in T1003042 :
    if 'X' in col and T1003042[col].std() == 0 :
        T1003042x_col.append(col)
print(len(T1003042x_col))
T1003042.drop(T1003042x_col, inplace = True, axis = 1)

T1003043x_col = []
for col in T1003043 :
    if 'X' in col and T1003043[col].std() == 0 :
        T1003043x_col.append(col)
print(len(T1003043x_col))
T1003043.drop(T1003043x_col, inplace = True, axis = 1)

T1003062x_col = []
for col in T1003062 :
    if 'X' in col and T1003062[col].std() == 0 :
        T1003062x_col.append(col)
print(len(T1003062x_col))
T1003062.drop(T1003062x_col, inplace = True, axis = 1)

T1003063x_col = []
for col in T1003063 :
    if 'X' in col and T1003063[col].std() == 0 :
        T1003063x_col.append(col)
print(len(T1003063x_col))
T1003063.drop(T1003063x_col, inplace = True, axis = 1)

269
233
152
150
124
209
124
217


In [20]:
T0503041 = T0503041.fillna(0)
T0503071 = T0503071.fillna(0)
T0103051 = T0103051.fillna(0)
T0103061 = T0103061.fillna(0)
T1003042 = T1003042.fillna(0)
T1003043 = T1003043.fillna(0)
T1003062 = T1003062.fillna(0)
T1003063 = T1003063.fillna(0)

In [29]:
T0503041.drop(['LINE'], axis = 1, inplace = True)
T0503071.drop(['LINE'], axis = 1, inplace = True)
T0103051.drop(['LINE'], axis = 1, inplace = True)
T0103061.drop(['LINE'], axis = 1, inplace = True)
T1003042.drop(['LINE'], axis = 1, inplace = True)
T1003043.drop(['LINE'], axis = 1, inplace = True) # *4
T1003062.drop(['LINE'], axis = 1, inplace = True)
T1003063.drop(['LINE'], axis = 1, inplace = True) # *4

In [30]:
T1003043 = pd.concat([T1003043, T1003043, T1003043, T1003043, T1003043]).reset_index()
T1003063 = pd.concat([T1003063, T1003063, T1003063, T1003063, T1003063]).reset_index()

# 1

In [31]:
T0503041_class = T0503041['Y_Class']
T0503041_target = T0503041['Y_Quality']
T0503041_data = T0503041.drop(['Y_Class', 'Y_Quality'], axis = 1)

In [32]:
T0503041c = T0503041_data.columns
T0503041scaler = RobustScaler()
T0503041scaled = T0503041scaler.fit_transform(T0503041_data)
T0503041_data = pd.DataFrame(T0503041scaled, columns = T0503041c)

T0503041_y_class = pd.DataFrame(T0503041_target, columns = ['Y_Quality'])
T0503041_idx = T0503041_y_class.index
T0503041_y_class = T0503041_y_class.reset_index()
T0503041_data = pd.concat([T0503041_data, T0503041_y_class], axis = 1)
T0503041_data = T0503041_data.set_index('index')
T0503041_data.drop(['Y_Quality'], axis = 1, inplace = True)
T0503041_data

PRODUCT_CODE     X_128     X_129  X_132  X_133  X_134  X_135     X_136  \
index                                                                           
0               0.0 -0.364170 -0.364234   -1.0   -1.0    0.0    0.0 -0.259277   
2               0.0 -0.364041 -0.364106   -1.0   -1.0    0.0    0.0 -0.259277   
4               0.0 -0.363913 -0.363977   -1.0   -1.0   -1.0    0.0 -0.259277   
6               0.0 -0.363721 -0.363785   -1.0   -1.0    0.0    0.0 -0.259277   
8               0.0 -0.363593 -0.363657   -1.0   -1.0    0.0    0.0 -0.259277   
...             ...       ...       ...    ...    ...    ...    ...       ...   
581             0.0 -0.104465 -0.104484   -1.0   -1.0    0.0    0.0 -0.255422   
582             0.0 -0.102285 -0.102304   -2.0   -1.0   -1.0    0.0 -0.255422   
583             0.0 -0.100875 -0.100893   -1.0   -1.0    0.0    0.0 -0.255422   
594             0.0  0.084399  0.084414   -1.0   -1.0    0.0    0.0 -0.112771   
595             0.0  0.084591  0.084606   -1.0   -1.0    0.0    0.0 -0.112771   

          X_137     X_138     X_139     X_140     X_141  X_143  X_144  X_145  \
index                                                                          
0     -0.259277 -0.264678 -0.075041 -0.091691 -0.213256    0.0   -1.0    0.0   
2     -0.259277 -0.264678 -0.016313 -0.063037  0.109510    0.0    0.0    0.0   
4     -0.259277 -0.264678 -0.055465 -0.080229 -0.293948    0.0   -1.0    0.0   
6     -0.259277 -0.264678 -0.042414 -0.080229 -1.561960    0.0   -2.0    0.0   
8     -0.259277 -0.264678 -0.048940 -0.085960  0.051873    0.0    0.0    0.0   
...         ...       ...       ...       ...       ...    ...    ...    ...   
581   -0.255422 -0.255053 -0.003263  0.000000 -0.737752    0.0   -1.0    0.0   
582   -0.255422 -0.255053 -0.048940 -0.017192 -0.051873    0.0   -1.0    0.0   
583   -0.255422 -0.255053 -0.035889 -0.022923 -0.870317    0.0    0.0    0.0   
594   -0.112771 -0.112608  0.316476  0.338109  0.219020    0.0    0.0    0.0   
595   -0.112771 -0.112608  0.309951  0.338109 -0.011527    0.0   -1.0    0.0   

          X_146     X_148     X_149  X_150  X_151  X_152  X_153  X_155  X_156  \
index                                                                           
0      0.000000  0.093023  0.090909   0.16   -1.0    0.0    0.0    0.0    0.0   
2      0.522293  0.000000  0.000000   0.08   -1.0    0.0   -0.1    0.0    0.0   
4     -0.318471  0.093023  0.000000   0.16    0.0    0.0   -0.1    0.0    0.1   
6     -3.789809  0.093023  0.000000   0.08    0.0    0.1    0.0   -1.0    0.0   
8      0.477707  0.093023  0.000000   0.16    0.0    0.0    0.0    0.0    0.0   
...         ...       ...       ...    ...    ...    ...    ...    ...    ...   
581   -1.987261  0.000000  0.000000   0.00    0.0    0.0    0.0   -1.0    0.0   
582    0.207006  0.093023  0.272727   0.08    0.0    0.0    0.0   -1.0    0.0   
583   -2.149682  0.093023  0.090909   0.08    0.0    0.0    0.0   -1.0    0.0   
594    0.105096 -0.279070 -0.272727  -0.32   -1.0    0.0    0.0    0.0    0.0   
595   -0.589172 -0.279070 -0.272727  -0.32    0.0    0.0    0.0    0.0    0.1   

       X_157  X_159  X_160  X_161  X_163  X_164  X_165  X_167  X_168  X_169  \
index                                                                         
0        0.0    0.0    0.0    0.0   -0.1    0.0    0.0   -1.0    0.0    0.0   
2        0.0    1.0    0.0    0.0   -0.1    0.0    0.0   -1.0    0.0    0.0   
4        0.1    0.0    0.0    0.0    0.0    1.0    1.0   -1.0    0.0    0.0   
6        0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
8        0.0    1.0    0.0    0.0    0.0    1.0    1.0    0.0    0.0    0.0   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
581      0.0    1.0    0.0    0.0    0.0    1.0    1.0    0.0    0.0    0.0   
582      0.0    1.0    0.0    0.0    0.0    1.0    1.0    0.0    0.1    0.0   
583      0.0    0.0    0.0    0.0    0.0    1.0    1.0   -1.0    0.0

In [33]:
#13
setup_rgs = setup(data = T0503041_data, target = T0503041_target, session_id = 4)
top5 = compare_models(sort='MSE', n_select=5)

Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [34]:
T05_total_models = []

T05_model_lightgbm = create_model('lightgbm', fold = 5)
T05_total_models.append(T05_model_lightgbm)

T05_model_rf = create_model('rf', fold = 5)
T05_total_models.append(T05_model_rf)

T05_model_gbr = create_model('gbr', fold = 5)
T05_total_models.append(T05_model_gbr)

T05_model_et = create_model('et', fold = 5)
T05_total_models.append(T05_model_et)

T05_model_catboost = create_model('catboost', fold = 5)
T05_total_models.append(T05_model_catboost)

T05_model_xgb = create_model('xgboost', fold = 5)
T05_total_models.append(T05_model_xgb)

T05_model_ada = create_model('ada', fold = 5)

T05_total_models.append(T05_model_ada)

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Initiated  . . . . . . . . . . . . . . . . . .            20:08:16
Status     . . . . . . . . . . . . . . . . . .     Fitting 5 Folds
Estimator  . . . . . . . . . . . . . . . . . .  CatBoost Regressor

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
T05_model_lightgbm = tune_model(T05_model_lightgbm, fold=5, choose_better = True)

T05_model_rf = tune_model(T05_model_rf, fold=5, choose_better = True)

T05_model_gbr = tune_model(T05_model_gbr, fold=5, choose_better = True)

T05_model_et = tune_model(T05_model_et, fold=5, choose_better = True)

T05_model_catboost = tune_model(T05_model_catboost, fold=5, choose_better = True)

T05_model_xgb = tune_model(T05_model_xgb, fold=5, choose_better = True)

T05_model_ada = tune_model(T05_model_ada, fold=5, choose_better = True)

In [ ]:
T05_tuned_models = [T05_model_lightgbm, T05_model_rf, T05_model_gbr, T05_model_et, T05_model_catboost, T05_model_xgb, T05_model_ada]

for model in T05_tuned_models:
    display(predict_model(model))

In [ ]:
T0503041_final_regression_model = finalize_model(T05_model_et)

T0503041_prediction = predict_model(T0503041_final_regression_model, data=T0503041_data)
T0503041_prediction

T0503041_datas = T0503041_prediction['prediction_label']
T0503041_datas = pd.DataFrame(T0503041_datas.values, columns = ['Y_Quality'])

T0503041y_idx= T0503041_class.index
T0503041y_idx = pd.DataFrame(T0503041y_idx, columns = ['index'])
T0503041_datas = pd.concat([T0503041y_idx, T0503041_datas], axis = 1)
T0503041_datas = T0503041_datas.set_index('index')
T0503041_datas

In [ ]:
from pycaret.classification import *
# 분류 패키지의 모든 모듈을 임포트한다.
exp_clf101 = setup(data=T0503041_datas, 
                   target=T0503041_class, 
                   fold=5,
                   session_id=4)
top5 = compare_models(sort='Accuracy', n_select=5)

In [ ]:
T05_total_models = []

T05_model_lightgbm = create_model('lightgbm', fold = 5)
T05_total_models.append(T05_model_lightgbm)

T05_model_rf = create_model('rf', fold = 5)
T05_total_models.append(T05_model_rf)

T05_model_gbc = create_model('gbc', fold = 5)
T05_total_models.append(T05_model_gbc)

T05_model_et = create_model('et', fold = 5)
T05_total_models.append(T05_model_et)

T05_model_catboost = create_model('catboost', fold = 5)
T05_total_models.append(T05_model_catboost)

T05_model_xgb = create_model('xgboost', fold = 5)
T05_total_models.append(T05_model_xgb)

T05_model_ada = create_model('ada', fold = 5)

T05_total_models.append(T05_model_ada)

In [ ]:
T05_model_lightgbm = tune_model(T05_model_lightgbm, fold=5, choose_better = True)

T05_model_rf = tune_model(T05_model_rf, fold=5, choose_better = True)

T05_model_gbc = tune_model(T05_model_gbc, fold=5, choose_better = True)

T05_model_et = tune_model(T05_model_et, fold=5, choose_better = True)

T05_model_catboost = tune_model(T05_model_catboost, fold=5, choose_better = True)

T05_model_xgb = tune_model(T05_model_xgb, fold=5, choose_better = True)

T05_model_ada = tune_model(T05_model_ada, fold=5, choose_better = True)

In [ ]:
T05_tuned_models = [T05_model_lightgbm, T05_model_rf, T05_model_gbc, T05_model_et, T05_model_catboost, T05_model_xgb, T05_model_ada]

for model in T05_tuned_models:
    display(predict_model(model))

In [ ]:
T0503041_final_classification_model = finalize_model(T0503041_model_et)

T0503041_prediction = predict_model(T0503041_final_classification_model, data=T0503041_datas)

T0503041_y_class = pd.DataFrame(T0503041_class, columns = ['Y_Class'])
T0503041_idx = T0503041_y_class.index
T0503041_y_class = T0503041_y_class.reset_index()
T0503041_prediction = pd.concat([T0503041_prediction, T0503041_y_class], axis = 1)
T0503041_prediction = T0503041_prediction.set_index('index')
T0503041_prediction.drop(['Y_Class'], axis = 1, inplace = True)

# 6

In [ ]:
T1003043_class = T1003043['Y_Class']
T1003043_target = T1003043['Y_Quality']
T1003043_data = T1003043.drop(['Y_Class', 'Y_Quality'], axis = 1)

In [ ]:
T1003043c = T1003043_data.columns
T1003043scaler = RobustScaler()
T1003043scaled = T1003043scaler.fit_transform(T1003043_data)
T1003043_data = pd.DataFrame(T1003043scaled, columns = T1003043c)

T1003043_y_class = pd.DataFrame(T1003043_target, columns = ['Y_Quality'])
T1003043_idx = T1003043_y_class.index
T1003043_y_class = T1003043_y_class.reset_index()
T1003043_data = pd.concat([T1003043_data, T1003043_y_class], axis = 1)
T1003043_data = T1003043_data.set_index('index')
T1003043_data.drop(['Y_Quality'], axis = 1, inplace = True)
T1003043_data

In [ ]:
from pycaret.regression import *
setup_rgs = setup(data = T1003043_data, target = T1003043_target, session_id = 4)
top5 = compare_models(sort='MSE', n_select=5)

In [ ]:
T01_total_models = []

T01_model_lightgbm = create_model('lightgbm', fold = 5)
T01_total_models.append(T01_model_lightgbm)

T01_model_rf = create_model('rf', fold = 5)
T01_total_models.append(T01_model_rf)

T01_model_gbr = create_model('gbr', fold = 5)
T01_total_models.append(T01_model_gbr)

T01_model_et = create_model('et', fold = 5)
T01_total_models.append(T01_model_et)

T01_model_catboost = create_model('catboost', fold = 5)
T01_total_models.append(T01_model_catboost)

T01_model_xgb = create_model('xgboost', fold = 5)
T01_total_models.append(T01_model_xgb)

T01_model_ada = create_model('ada', fold = 5)
T01_total_models.append(T01_model_ada)

In [ ]:
T01_model_lightgbm = tune_model(T01_model_lightgbm, fold=5, choose_better = True)

T01_model_rf = tune_model(T01_model_rf, fold=5, choose_better = True)

T01_model_gbr = tune_model(T01_model_gbr, fold=5, choose_better = True)

T01_model_et = tune_model(T01_model_et, fold=5, choose_better = True)

T01_model_catboost = tune_model(T01_model_catboost, fold=5, choose_better = True)

T01_model_xgb = tune_model(T01_model_xgb, fold=5, choose_better = True)

T01_model_ada = tune_model(T01_model_ada, fold=5, choose_better = True)

In [ ]:
T01_tuned_models = [T01_model_lightgbm, T01_model_rf, T01_model_gbr, T01_model_et, T01_model_catboost, T01_model_xgb, T01_model_ada]

for model in T01_tuned_models:
    display(predict_model(model))

In [ ]:
T1003043_final_regression_model = finalize_model(T1003043_model_et)

T1003043_prediction = predict_model(T1003043_final_regression_model, data=T1003043_data)

T1003043_datas = T1003043_prediction['prediction_label']
T1003043_datas = pd.DataFrame(T1003043_datas.values, columns = ['Y_Quality'])

T1003043y_idx= T1003043_class.index
T1003043y_idx = pd.DataFrame(T1003043y_idx, columns = ['index'])
T1003043_datas = pd.concat([T1003043y_idx, T1003043_datas], axis = 1)
T1003043_datas = T1003043_datas.set_index('index')
T1003043_datas

In [ ]:
from pycaret.classification import *
# 분류 패키지의 모든 모듈을 임포트한다.
exp_clf101 = setup(data=T1003043_datas, 
                   target=T1003043_class, 
                   fold=5,
                   session_id=414)
top5 = compare_models(sort='Accuracy', n_select=5)

In [ ]:
T01_total_models = []

T01_model_lightgbm = create_model('lightgbm', fold = 5)
T01_total_models.append(T01_model_lightgbm)

T01_model_rf = create_model('rf', fold = 5)
T01_total_models.append(T01_model_rf)

T01_model_gbc = create_model('gbc', fold = 5)
T01_total_models.append(T01_model_gbc)

T01_model_et = create_model('et', fold = 5)
T01_total_models.append(T01_model_et)

T01_model_catboost = create_model('catboost', fold = 5)
T01_total_models.append(T01_model_catboost)

T01_model_xgb = create_model('xgboost', fold = 5)
T01_total_models.append(T01_model_xgb)

T01_model_ada = create_model('ada', fold = 5)
T01_total_models.append(T01_model_ada)

In [ ]:
T01_model_lightgbm = tune_model(T01_model_lightgbm, fold=5, choose_better = True)

T01_model_rf = tune_model(T01_model_rf, fold=5, choose_better = True)

T01_model_gbc = tune_model(T01_model_gbc, fold=5, choose_better = True)

T01_model_et = tune_model(T01_model_et, fold=5, choose_better = True)

T01_model_catboost = tune_model(T01_model_catboost, fold=5, choose_better = True)

T01_model_xgb = tune_model(T01_model_xgb, fold=5, choose_better = True)

T01_model_ada = tune_model(T01_model_ada, fold=5, choose_better = True)

In [ ]:
T01_tuned_models = [T01_model_lightgbm, T01_model_rf, T01_model_gbc, T01_model_et, T01_model_catboost, T01_model_xgb, T01_model_ada]

for model in T01_tuned_models:
    display(predict_model(model))

In [ ]:
T1003043_final_classification_model = finalize_model(T1003043_model_gbc)

T1003043_prediction = predict_model(T1003043_final_classification_model, data=T1003043_datas)

T1003043_prediction = pd.DataFrame(T1003043_prediction, columns = ['prediction_label'])

T1003043_y_class = pd.DataFrame(T1003043_class, columns = ['Y_Class'])
T1003043_idx = T1003043_y_class.index
T1003043_y_class = T1003043_y_class.reset_index()
T1003043_prediction = pd.concat([T1003043_prediction, T1003043_y_class], axis = 1)
T1003043_prediction = T1003043_prediction.set_index('index')
T1003043_prediction.drop(['Y_Class'], axis = 1, inplace = True)

# 7

In [ ]:
T1003062_class = T1003062['Y_Class']
T1003062_target = T1003062['Y_Quality']
T1003062_data = T1003062.drop(['Y_Class', 'Y_Quality'], axis = 1)

In [ ]:
T1003062c = T1003062_data.columns
T1003062scaler = RobustScaler()
T1003062scaled = T1003062scaler.fit_transform(T1003062_data)
T1003062_data = pd.DataFrame(T1003062scaled, columns = T1003062c)

T1003062_y_class = pd.DataFrame(T1003062_target, columns = ['Y_Quality'])
T1003062_idx = T1003062_y_class.index
T1003062_y_class = T1003062_y_class.reset_index()
T1003062_data = pd.concat([T1003062_data, T1003062_y_class], axis = 1)
T1003062_data = T1003062_data.set_index('index')
T1003062_data.drop(['Y_Quality'], axis = 1, inplace = True)
T1003062_data

In [ ]:
#13
setup_rgs = setup(data = T1003062_data, target = T1003062_target, session_id = 4)
top5 = compare_models(sort='MSE', n_select=5)

In [ ]:
T10_total_models = []

T10_model_lightgbm = create_model('lightgbm', fold = 5)
T10_total_models.append(T10_model_lightgbm)

T10_model_rf = create_model('rf', fold = 5)
T10_total_models.append(T10_model_rf)

T10_model_gbr = create_model('gbr', fold = 5)
T10_total_models.append(T10_model_gbr)

T10_model_et = create_model('et', fold = 5)
T10_total_models.append(T10_model_et)

T10_model_catboost = create_model('catboost', fold = 5)
T10_total_models.append(T10_model_catboost)

T10_model_xgb = create_model('xgboost', fold = 5)
T10_total_models.append(T10_model_xgb)

T10_model_ada = create_model('ada', fold = 5)

T10_total_models.append(T10_model_ada)

In [ ]:
T10_model_lightgbm = tune_model(T10_model_lightgbm, fold=5, choose_better = True)

T10_model_rf = tune_model(T10_model_rf, fold=5, choose_better = True)

T10_model_gbr = tune_model(T10_model_gbr, fold=5, choose_better = True)

T10_model_et = tune_model(T10_model_et, fold=5, choose_better = True)

T10_model_catboost = tune_model(T10_model_catboost, fold=5, choose_better = True)

T10_model_xgb = tune_model(T10_model_xgb, fold=5, choose_better = True)

T10_model_ada = tune_model(T10_model_ada, fold=5, choose_better = True)

In [ ]:
T10_tuned_models = [T10_model_lightgbm, T10_model_rf, T10_model_gbr, T10_model_et, T10_model_catboost, T10_model_xgb, T10_model_ada]

for model in T10_tuned_models:
    display(predict_model(model))

In [ ]:
T1003062_final_regression_model = finalize_model(T10_model_et)

T1003062_prediction = predict_model(T1003062_final_regression_model, data=T1003062_data)
T1003062_prediction

T1003062_datas = T1003062_prediction['prediction_label']
T1003062_datas = pd.DataFrame(T1003062_datas.values, columns = ['Y_Quality'])

T1003062y_idx= T1003062_class.index
T1003062y_idx = pd.DataFrame(T1003062y_idx, columns = ['index'])
T1003062_datas = pd.concat([T1003062y_idx, T1003062_datas], axis = 1)
T1003062_datas = T1003062_datas.set_index('index')
T1003062_datas

In [ ]:
from pycaret.classification import *
# 분류 패키지의 모든 모듈을 임포트한다.
exp_clf101 = setup(data=T1003062_datas, 
                   target=T1003062_class, 
                   fold=5,
                   session_id=4)
top5 = compare_models(sort='Accuracy', n_select=5)

In [ ]:
T10_total_models = []

T10_model_lightgbm = create_model('lightgbm', fold = 5)
T10_total_models.append(T10_model_lightgbm)

T10_model_rf = create_model('rf', fold = 5)
T10_total_models.append(T10_model_rf)

T10_model_gbc = create_model('gbc', fold = 5)
T10_total_models.append(T10_model_gbc)

T10_model_et = create_model('et', fold = 5)
T10_total_models.append(T10_model_et)

T10_model_catboost = create_model('catboost', fold = 5)
T10_total_models.append(T10_model_catboost)

T10_model_xgb = create_model('xgboost', fold = 5)
T10_total_models.append(T10_model_xgb)

T10_model_ada = create_model('ada', fold = 5)

T10_total_models.append(T10_model_ada)

In [ ]:
T10_model_lightgbm = tune_model(T10_model_lightgbm, fold=5, choose_better = True)

T10_model_rf = tune_model(T10_model_rf, fold=5, choose_better = True)

T10_model_gbc = tune_model(T10_model_gbc, fold=5, choose_better = True)

T10_model_et = tune_model(T10_model_et, fold=5, choose_better = True)

T10_model_catboost = tune_model(T10_model_catboost, fold=5, choose_better = True)

T10_model_xgb = tune_model(T10_model_xgb, fold=5, choose_better = True)

T10_model_ada = tune_model(T10_model_ada, fold=5, choose_better = True)

In [ ]:
T10_tuned_models = [T10_model_lightgbm, T10_model_rf, T10_model_gbc, T10_model_et, T10_model_catboost, T10_model_xgb, T10_model_ada]

for model in T10_tuned_models:
    display(predict_model(model))

In [1]:
T1003062_final_classification_model = finalize_model(T1003062_model_et)

T1003062_prediction = predict_model(T1003062_final_classification_model, data=T1003062_datas)

T1003062_prediction = pd.DataFrame(T1003062_prediction, columns = ['prediction_label'])

T1003062_y_class = pd.DataFrame(T1003062_class, columns = ['Y_Class'])
T1003062_idx = T1003062_y_class.index
T1003062_y_class = T1003062_y_class.reset_index()
T1003062_prediction = pd.concat([T1003062_prediction, T1003062_y_class], axis = 1)
T1003062_prediction = T1003062_prediction.set_index('index')
T1003062_prediction.drop(['Y_Class'], axis = 1, inplace = True)

NameError: name 'finalize_model' is not defined

# 8

In [ ]:
T1003063_class = T1003063['Y_Class']
T1003063_target = T1003063['Y_Quality']
T1003063_data = T1003063.drop(['Y_Class', 'Y_Quality'], axis = 1)

In [ ]:
T1003063c = T1003063_data.columns
T1003063scaler = RobustScaler()
T1003063scaled = T1003063scaler.fit_transform(T1003063_data)
T1003063_data = pd.DataFrame(T1003063scaled, columns = T1003063c)

T1003063_y_class = pd.DataFrame(T1003063_target, columns = ['Y_Quality'])
T1003063_idx = T1003063_y_class.index
T1003063_y_class = T1003063_y_class.reset_index()
T1003063_data = pd.concat([T1003063_data, T1003063_y_class], axis = 1)
T1003063_data = T1003063_data.set_index('index')
T1003063_data.drop(['Y_Quality'], axis = 1, inplace = True)
T1003063_data

In [ ]:
#13
setup_rgs = setup(data = T1003063_data, target = T1003063_target, session_id = 4)
top5 = compare_models(sort='MSE', n_select=5)

In [ ]:
T10_total_models = []

T10_model_lightgbm = create_model('lightgbm', fold = 5)
T10_total_models.append(T10_model_lightgbm)

T10_model_rf = create_model('rf', fold = 5)
T10_total_models.append(T10_model_rf)

T10_model_gbr = create_model('gbr', fold = 5)
T10_total_models.append(T10_model_gbr)

T10_model_et = create_model('et', fold = 5)
T10_total_models.append(T10_model_et)

T10_model_catboost = create_model('catboost', fold = 5)
T10_total_models.append(T10_model_catboost)

T10_model_xgb = create_model('xgboost', fold = 5)
T10_total_models.append(T10_model_xgb)

T10_model_ada = create_model('ada', fold = 5)

T10_total_models.append(T10_model_ada)

In [ ]:
T10_model_lightgbm = tune_model(T10_model_lightgbm, fold=5, choose_better = True)

T10_model_rf = tune_model(T10_model_rf, fold=5, choose_better = True)

T10_model_gbr = tune_model(T10_model_gbr, fold=5, choose_better = True)

T10_model_et = tune_model(T10_model_et, fold=5, choose_better = True)

T10_model_catboost = tune_model(T10_model_catboost, fold=5, choose_better = True)

T10_model_xgb = tune_model(T10_model_xgb, fold=5, choose_better = True)

T10_model_ada = tune_model(T10_model_ada, fold=5, choose_better = True)

In [ ]:
T10_tuned_models = [T10_model_lightgbm, T10_model_rf, T10_model_gbr, T10_model_et, T10_model_catboost, T10_model_xgb, T10_model_ada]

for model in T10_tuned_models:
    display(predict_model(model))

In [ ]:
T1003063_final_regression_model = finalize_model(T10_model_et)

T1003063_prediction = predict_model(T1003063_final_regression_model, data=T1003063_data)
T1003063_prediction

T1003063_datas = T1003063_prediction['prediction_label']
T1003063_datas = pd.DataFrame(T1003063_datas.values, columns = ['Y_Quality'])

T1003063y_idx= T1003063_class.index
T1003063y_idx = pd.DataFrame(T1003063y_idx, columns = ['index'])
T1003063_datas = pd.concat([T1003063y_idx, T0503041_datas], axis = 1)
T1003063_datas = T1003063_datas.set_index('index')
T1003063_datas

In [ ]:
from pycaret.classification import *
# 분류 패키지의 모든 모듈을 임포트한다.
exp_clf101 = setup(data=T1003063_datas, 
                   target=T1003063_class, 
                   fold=5,
                   session_id=4)
top5 = compare_models(sort='Accuracy', n_select=5)

In [ ]:
T10_total_models = []

T10_model_lightgbm = create_model('lightgbm', fold = 5)
T10_total_models.append(T10_model_lightgbm)

T10_model_rf = create_model('rf', fold = 5)
T10_total_models.append(T10_model_rf)

T10_model_gbc = create_model('gbc', fold = 5)
T10_total_models.append(T10_model_gbc)

T10_model_et = create_model('et', fold = 5)
T10_total_models.append(T10_model_et)

T10_model_catboost = create_model('catboost', fold = 5)
T10_total_models.append(T10_model_catboost)

T10_model_xgb = create_model('xgboost', fold = 5)
T10_total_models.append(T10_model_xgb)

T10_model_ada = create_model('ada', fold = 5)

T10_total_models.append(T10_model_ada)

In [ ]:
T10_model_lightgbm = tune_model(T10_model_lightgbm, fold=5, choose_better = True)

T10_model_rf = tune_model(T10_model_rf, fold=5, choose_better = True)

T10_model_gbc = tune_model(T10_model_gbc, fold=5, choose_better = True)

T10_model_et = tune_model(T10_model_et, fold=5, choose_better = True)

T10_model_catboost = tune_model(T10_model_catboost, fold=5, choose_better = True)

T10_model_xgb = tune_model(T10_model_xgb, fold=5, choose_better = True)

T10_model_ada = tune_model(T10_model_ada, fold=5, choose_better = True)

In [ ]:
T10_tuned_models = [T10_model_lightgbm, T10_model_rf, T10_model_gbc, T10_model_et, T10_model_catboost, T10_model_xgb, T10_model_ada]

for model in T10_tuned_models:
    display(predict_model(model))

In [ ]:
T1003063_final_classification_model = finalize_model(T1003063_model_et)

T1003063_prediction = predict_model(T1003063_final_classification_model, data=T1003063_datas)

T1003063_prediction = pd.DataFrame(T1003063_prediction, columns = ['prediction_label'])

T1003063_y_class = pd.DataFrame(T1003063_class, columns = ['Y_Class'])
T1003063_idx = T1003063_y_class.index
T1003063_y_class = T1003063_y_class.reset_index()
T1003063_prediction = pd.concat([T1003063_prediction, T1003063_y_class], axis = 1)
T1003063_prediction = T1003063_prediction.set_index('index')
T1003063_prediction.drop(['Y_Class'], axis = 1, inplace = True)

In [ ]:
TT0503041.drop(['LINE'], axis = 1, inplace = True)
TT0503071.drop(['LINE'], axis = 1, inplace = True)
TT0103051.drop(['LINE'], axis = 1, inplace = True)
TT0103061.drop(['LINE'], axis = 1, inplace = True)
TT1003042.drop(['LINE'], axis = 1, inplace = True)
TT1003043.drop(['LINE'], axis = 1, inplace = True)
TT1003062.drop(['LINE'], axis = 1, inplace = True)
TT1003063.drop(['LINE'], axis = 1, inplace = True)

In [ ]:
TT05_y_class = pd.DataFrame(TT05, columns = ['PRODUCT_CODE'])
TT05_idx = TT05_y_class.index
TT05_y_class = TT05_y_class.reset_index()
TT05_prediction = pd.concat([TT05_prediction, TT05_y_class], axis = 1)
TT05_prediction = TT05_prediction.set_index('index')
TT05_prediction.drop(['PRODUCT_CODE'], axis = 1, inplace = True)

TT01_y_class = pd.DataFrame(TT01, columns = ['PRODUCT_CODE'])
TT01_idx = TT01_y_class.index
TT01_y_class = TT01_y_class.reset_index()
TT01_prediction = pd.concat([TT01_prediction, TT01_y_class], axis = 1)
TT01_prediction = TT01_prediction.set_index('index')
TT01_prediction.drop(['PRODUCT_CODE'], axis = 1, inplace = True)

TT10_y_class = pd.DataFrame(TT10, columns = ['PRODUCT_CODE'])
TT10_idx = TT10_y_class.index
TT10_y_class = TT10_y_class.reset_index()
TT10_prediction = pd.concat([TT10_prediction, TT10_y_class], axis = 1)
TT10_prediction = TT10_prediction.set_index('index')
TT10_prediction.drop(['PRODUCT_CODE'], axis = 1, inplace = True)

In [ ]:
TT05_datas = TT05_prediction['prediction_label']
TT05_datas = pd.DataFrame(TT05_datas.values, columns = ['Y_Quality'])

TT05y_idx= TT05_prediction.index
TT05y_idx = pd.DataFrame(TT05y_idx, columns = ['index'])
TT05_datas = pd.concat([TT05y_idx, TT05_datas], axis = 1)
TT05_datas = TT05_datas.set_index('index')

TT01_datas = TT01_prediction['prediction_label']
TT01_datas = pd.DataFrame(TT01_datas.values, columns = ['Y_Quality'])

TT01y_idx= TT01_prediction.index
TT01y_idx = pd.DataFrame(TT01y_idx, columns = ['index'])
TT01_datas = pd.concat([TT01y_idx, TT01_datas], axis = 1)
TT01_datas = TT01_datas.set_index('index')

TT10_datas = TT10_prediction['prediction_label']
TT10_datas = pd.DataFrame(TT10_datas.values, columns = ['Y_Quality'])

TT10y_idx= TT10_prediction.index
TT10y_idx = pd.DataFrame(TT10y_idx, columns = ['index'])
TT10_datas = pd.concat([TT10y_idx, TT10_datas], axis = 1)
TT10_datas = TT10_datas.set_index('index')

In [ ]:
TT05_idxf = TT05_datas.index
TT05_p = pd.DataFrame(TT05_idxf, columns = ['index'])
TT05_predictions = pd.concat([TT05_p, TT05_predictions], axis = 1)
TT05_predictions = TT05_predictions.set_index('index')

TT01_idxf = TT01_datas.index
TT01_p = pd.DataFrame(TT01_idxf, columns = ['index'])
TT01_predictions = pd.concat([TT01_p, TT01_predictions], axis = 1)
TT01_predictions = TT01_predictions.set_index('index')

TT10_idxf = TT10_datas.index
TT10_p = pd.DataFrame(TT10_idxf, columns = ['index'])
TT10_predictions = pd.concat([TT10_p, TT10_predictions], axis = 1)
TT10_predictions = TT10_predictions.set_index('index')